# Buddhist Wisdom RAG System

This notebook implements a Retrieval Augmented Generation (RAG) system that provides philosophical advice based on Buddhist teachings. It uses:
- A PDF document containing Buddhist philosophical text
- LangChain for RAG implementation
- OpenAI's GPT model for generating compassionate, contextual responses
- Vector storage for semantic search

The system will:
1. Load and process Buddhist teachings from PDF
2. Create embeddings for semantic search
3. Take user queries about emotional challenges
4. Provide thoughtful advice grounded in Buddhist philosophy

## 1. Setup and Dependencies

First, we'll import the required libraries:
- `PyPDF2` for reading PDF files
- `langchain` components for RAG
- `dotenv` for environment variables
- Custom prompt templates for compassionate responses

In [8]:
from __future__ import annotations

import os
from typing import List

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore

# Load environment variables
load_dotenv()

# Configure API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError(
        "OPENROUTER_API_KEY not found in .env file"
    )

## 2. Load and Process Text Document

We'll now:
1. Load the Buddhist teachings from a text file
2. Split it into manageable chunks
3. Create Document objects for the RAG system

The text splitter is configured to preserve context while creating chunks that are small enough for effective retrieval.

In [9]:
# Load text file
with open("buddhist.txt", "r") as file:
    text_content = file.read()

# Create initial document
documents = [Document(page_content=text_content)]

# Configure text splitter for optimal chunk size
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
)

# Split documents into chunks
chunks = text_splitter.split_documents(documents)
print(f"Split text into {len(chunks)} chunks")

# Preview first chunk
if chunks:
    print("\nSample chunk content:")
    print("=" * 40)
    print(chunks[0].page_content[:200], "...")

Split text into 12 chunks

Sample chunk content:
The Noble Eightfold Path and the Nature of Suffering

In Buddhist philosophy, anxiety and suffering (dukkha) are understood as natural parts of human existence. The Buddha taught that while suffering  ...


## 3. Create Vector Store

Now we'll:
1. Initialize the embeddings model
2. Create embeddings for all chunks
3. Store them in a vector store for similarity search

We'll use OpenAI's embedding model for high-quality semantic search.

In [11]:
# Initialize the embedding model
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-large",  # Use a standard embedding model name
    openai_api_key=api_key,
    openai_api_base="https://api.openai.com/v1",
)

# Create embeddings and store them in a vector store
vectorstore = InMemoryVectorStore.from_documents(
    chunks,
    embeddings_model
)

## 4. Create Response Generator

We'll create a chain that:
1. Takes a user's emotional concern
2. Retrieves relevant Buddhist wisdom
3. Generates a compassionate, thoughtful response

The prompt is designed to:
- Show empathy for the user's feelings
- Ground advice in Buddhist teachings
- Offer practical suggestions
- Maintain a warm, supportive tone

In [15]:
# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-5-mini",  # or another suitable model
)

# Create a thoughtful prompt template
template = """You are a wise and compassionate Buddhist teacher. Using the Buddhist teachings provided below, 
offer gentle guidance to help someone who is experiencing emotional difficulty. Your response should be warm, 
understanding, and grounded in Buddhist wisdom while being accessible to someone who might be new to Buddhist concepts.

Context from Buddhist teachings:
{context}

Person's concern: {question}

Please provide guidance that:
1. Acknowledges their feelings with compassion
2. Offers relevant wisdom from Buddhist teachings
3. Suggests practical steps they can take
4. Ends with words of encouragement

Response:"""

prompt = ChatPromptTemplate.from_template(template)

def get_buddhist_guidance(question: str, num_chunks: int = 3) -> str:
    """Generate Buddhist-inspired guidance for an emotional concern."""
    # Retrieve relevant passages
    context_chunks = vectorstore.similarity_search(question, k=num_chunks)
    context = "\n\n".join(doc.page_content for doc in context_chunks)
    
    # Generate response
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({"context": context, "question": question})
    return response


## 5. Interactive Guidance System

Now you can use the cell below to ask for guidance about any emotional concern. The system will:
1. Find relevant Buddhist teachings
2. Generate thoughtful, compassionate advice
3. Provide practical suggestions based on Buddhist wisdom

Try asking about feelings like:
- Anxiety
- Stress
- Uncertainty
- Loneliness
- Frustration

In [ ]:
# Interactive cell for seeking guidance
question = "I am feeling anxious about my future"  # Replace with your concern
print("Your concern:", question)
print("\nBuddhist Wisdom & Guidance:")
print("=" * 60)
print(get_buddhist_guidance(question))

Your concern: I am feeling anxious

Buddhist Wisdom & Guidance:
I’m sorry you’re feeling anxious — that’s a difficult place to be, and it matters that you noticed it. Thank you for being honest with yourself. I’m with you in this moment.

A teaching from the Buddha that may be helpful is that much of our suffering comes from resisting what is happening now or getting pulled into worries about the future or regrets about the past. Anxiety often feels like being swept away by thoughts. But we can learn to be a gentle observer of those thoughts and sensations, like watching clouds pass across the sky.

Here are some simple, practical steps you can try now, grounded in mindfulness:

1. Pause and come back to your body  
   - Sit or stand comfortably. Put both feet on the ground or feel the chair supporting you. Take one slow, intentional breath in and out. This small pause helps you leave the stream of worry for a moment.

2. Notice physical sensations (without judging them)  
   - Ask you